<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/Fragility_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Bi-directional Fragility Index (Walsh-compliant, symmetric) for a 2×2 table
# Layout:
#   |  Events | Non-events |
# A |    a    |     b      |
# B |    c    |     d      |

# Install SciPy if needed
try:
    import scipy
except ImportError:
    !pip -q install scipy

from scipy.stats import fisher_exact

ALPHA = 0.05  # significance threshold
# Definition here follows user spec: p <= ALPHA = significant; p > ALPHA = non-significant

def fisher_p(a, b, c, d):
    _, p = fisher_exact([[a, b], [c, d]], alternative="two-sided")
    return p

def significant(p, alpha=ALPHA):
    return p <= alpha

def step(a, b, c, d, move):
    """
    move in {"A_up","A_down","B_up","B_down"}:
      *_up   : non-event -> event  in that arm
      *_down : event     -> non-event in that arm
    Row totals fixed; column totals change.
    Returns new tuple or None if move impossible.
    """
    if move == "A_up":
        if b <= 0: return None
        return (a+1, b-1, c,   d)
    if move == "A_down":
        if a <= 0: return None
        return (a-1, b+1, c,   d)
    if move == "B_up":
        if d <= 0: return None
        return (a,   b,   c+1, d-1)
    if move == "B_down":
        if c <= 0: return None
        return (a,   b,   c-1, d+1)
    raise ValueError("Unknown move")

def simulate_to_target(a, b, c, d, target_sig, move, max_iter=10**6):
    """
    Apply the same move repeatedly until significance state flips to target_sig.
    Returns steps, final_p, or (None, final_p) if not attainable with this move.
    """
    A,B,C,D = a,b,c,d
    p0 = fisher_p(A,B,C,D)
    steps = 0
    for _ in range(max_iter):
        nxt = step(A,B,C,D, move)
        if nxt is None:
            return None, fisher_p(A,B,C,D)  # not attainable along this path
        A,B,C,D = nxt
        steps += 1
        p_now = fisher_p(A,B,C,D)
        if significant(p_now) == target_sig:
            return steps, p_now
    return None, fisher_p(A,B,C,D)

def compute_fi(a, b, c, d, alpha=ALPHA):
    # Validate inputs
    for x in (a,b,c,d):
        if not (isinstance(x, int) and x >= 0):
            raise ValueError("Cells a,b,c,d must be non-negative integers.")
    p0 = fisher_p(a,b,c,d)
    base_sig = significant(p0, alpha)
    target_sig = not base_sig  # flip significance state

    # Try all four monotone strategies; pick minimal steps that succeed
    candidates = []
    for mv in ("A_up","A_down","B_up","B_down"):
        steps, p_final = simulate_to_target(a,b,c,d, target_sig, mv)
        if steps is not None:
            candidates.append((steps, mv, p_final))

    if not candidates:
        return {
            "FI": None,
            "baseline_p": p0,
            "baseline_state": "significant" if base_sig else "non-significant",
            "target_state": "non-significant" if base_sig else "significant",
            "move": None,
            "final_p": p0,
            "note": "Not attainable with allowed one-at-a-time flips."
        }

    steps, mv, p_final = min(candidates, key=lambda x: x[0])

    # Describe direction of flips for clarity
    move_desc = {
        "A_up":   "Arm A: non-events → events",
        "A_down": "Arm A: events → non-events",
        "B_up":   "Arm B: non-events → events",
        "B_down": "Arm B: events → non-events",
    }[mv]

    return {
        "FI": steps,
        "baseline_p": p0,
        "baseline_state": "significant" if base_sig else "non-significant",
        "target_state": "non-significant" if base_sig else "significant",
        "move": mv,
        "move_description": move_desc,
        "final_p": p_final,
    }

def main():
    print("Enter 2×2 table cells as integers.")
    a = int(input("a (Arm A events): ").strip())
    b = int(input("b (Arm A non-events): ").strip())
    c = int(input("c (Arm B events): ").strip())
    d = int(input("d (Arm B non-events): ").strip())

    res = compute_fi(a,b,c,d, alpha=ALPHA)
    print("\n--- Fragility Index (Bidirectional) ---")
    print(f"Baseline Fisher p = {res['baseline_p']:.6f}  →  {res['baseline_state']}")
    print(f"Target state: {res['target_state']} (p {'<=' if res['target_state']=='significant' else '>'} {ALPHA:.2f})")

    if res["FI"] is None:
        print("FI = Not attainable with allowed flips.")
        print(res["note"])
        return

    print(f"FI = {res['FI']} step(s)")
    print(f"Flip rule used: {res['move_description']}")
    print(f"Final Fisher p = {res['final_p']:.6f}")

if __name__ == "__main__":
    main()


Enter 2×2 table cells as integers.
a (Arm A events): 13
b (Arm A non-events): 14
c (Arm B events): 52
d (Arm B non-events): 12

--- Fragility Index (Bidirectional) ---
Baseline Fisher p = 0.002275  →  significant
Target state: non-significant (p > 0.05)
FI = 4 step(s)
Flip rule used: Arm A: non-events → events
Final Fisher p = 0.105660
